# 7.0 简介

核心内容：本章介绍使用pandas和NumPy处理日期时间数据的方法，包括转换、时区处理、筛选、特征提取、时间差计算、滞后特征、滚动窗口和缺失值处理等

# 7.1 把字符串转换为日期

问题描述：将字符串格式的日期转换为datetime类型

In [1]:
# 加载库
import numpy as np
import pandas as pd

# 创建字符串
date_strings = np.array(['03-04-2005 11:35 PM',
                         '23-05-2010 12:01 AM',
                         '04-09-2009 09:09 PM'])

# 转换成datetime
pd.to_datetime(date_strings, format='%d-%m-%Y %I:%M %p')
# 输出:
# DatetimeIndex(['2005-04-03 23:35:00', '2010-05-23 00:01:00',
#                '2009-09-04 21:09:00'], dtype='datetime64[ns]', freq=None)

DatetimeIndex(['2005-04-03 23:35:00', '2010-05-23 00:01:00',
               '2009-09-04 21:09:00'],
              dtype='datetime64[ns]', freq=None)

错误处理：

In [2]:
# 使用errors参数处理错误（将错误值设为NaT）
pd.to_datetime(date_strings, errors='coerce')

C:\Users\24677\AppData\Local\Temp\ipykernel_32164\4290105888.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(date_strings, errors='coerce')


DatetimeIndex(['2005-03-04 23:35:00', '2010-05-23 00:01:00',
               '2009-04-09 21:09:00'],
              dtype='datetime64[ns]', freq=None)

常用格式化代码：

| 代码   | 描述                   | 例子   |
| ---- | -------------------- | ---- |
| `%Y` | 完整的年份                | 2001 |
| `%m` | 月，首位空缺时需用0填充         | 04   |
| `%d` | 日，首位空缺时需用0填充         | 09   |
| `%I` | 小时（12小时制），首位空缺时需用0填充 | 09   |
| `%M` | 分，首位空缺时需用0填充         | 02   |
| `%S` | 秒，首位空缺时需用0填充         | 05   |
| `%p` | AM（上午）或PM（下午）        | AM   |


# 7.2 处理时区

In [4]:
# 加载库
import pandas as pd

# 创建datetime
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='ME'))

# 设置时区（UTC）
dates_with_tz = dates.dt.tz_localize('UTC')

# 转换时区
dates_with_tz.dt.tz_convert('Africa/Abidjan')
# 输出:
# 0   2002-02-28 00:00:00+00:00
# 1   2002-03-31 00:00:00+00:00
# 2   2002-04-30 00:00:00+00:00
# dtype: datetime64[ns, Africa/Abidjan]

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

查看所有时区：

In [5]:
# 加载库
from pytz import all_timezones

# 查看前两个时区
all_timezones[0:2]
# 输出: ['Africa/Abidjan', 'Africa/Accra']

['Africa/Abidjan', 'Africa/Accra']

# 7.3选择日期和时间

问题描述：从日期向量中筛选特定范围的日期

In [7]:
# 加载库
import pandas as pd

# 创建数据帧
dataframe = pd.DataFrame()

# 创建datetime
dataframe['date'] = pd.date_range('1/1/2001', periods=100000, freq='h')

# 筛选出两个日期之间的观察值
dataframe[(dataframe['date'] > '2002-1-1 01:00:00') &
          (dataframe['date'] <= '2002-1-1 04:00:00')]

,date
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


# 7.4将日期数据切分成多个特征

问题描述：从日期中提取年、月、日、小时等特征

In [9]:
# 加载库
import pandas as pd

# 创建数据帧
dataframe = pd.DataFrame()

# 创建5个日期
dataframe['date'] = pd.date_range('1/1/2001', periods=5, freq='ME')

# 提取时间特征
dataframe['year'] = dataframe['date'].dt.year
dataframe['month'] = dataframe['date'].dt.month
dataframe['day'] = dataframe['date'].dt.day
dataframe['hour'] = dataframe['date'].dt.hour
dataframe['minute'] = dataframe['date'].dt.minute

# 查看结果
dataframe.head()

,date,year,month,day,hour,minute
0,2001-01-31,2001,1,31,0,0
1,2001-02-28,2001,2,28,0,0
2,2001-03-31,2001,3,31,0,0
3,2001-04-30,2001,4,30,0,0
4,2001-05-31,2001,5,31,0,0


# 7.5 计算两个日期之间的时间差

问题描述：计算两个日期特征之间的时间差

In [10]:
# 加载库
import pandas as pd

# 创建数据帧
dataframe = pd.DataFrame()

# 创建两个datetime特征
dataframe['arrived'] = pd.date_range('1/1/2001', periods=5, freq='D')
dataframe['left'] = pd.date_range('1/1/2001 01:00:00', periods=5, freq='D')

# 计算时间差
dataframe['left'] - dataframe['arrived']
# 输出:
# 0   0 days 01:00:00
# 1   0 days 01:00:00
# 2   0 days 01:00:00
# 3   0 days 01:00:00
# 4   0 days 01:00:00
# dtype: timedelta64[ns]

0   0 days 01:00:00
1   0 days 01:00:00
2   0 days 01:00:00
3   0 days 01:00:00
4   0 days 01:00:00
dtype: timedelta64[ns]

只提取天数：

In [11]:
# 计算天数差
(dataframe['left'] - dataframe['arrived']).dt.days
# 输出: 0    0
#       1    0
#       2    0
#       3    0
#       4    0
#       dtype: int64

0    0
1    0
2    0
3    0
4    0
dtype: int64

# 7.6 对一周内的各天进行编码

问题描述：将日期转换为星期几的名称或数字

In [19]:
# 加载库
import pandas as pd

# 创建日期
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='ME'))

# 查看星期几名称
dates.dt.day_name()



0    Thursday
1      Sunday
2     Tuesday
dtype: object

In [18]:
# 查看星期几数字（0=周一，6=周日）
dates.dt.weekday


0    3
1    6
2    1
dtype: int32

# 7.7 创建一个滞后的特征

问题描述：创建滞后n个时间段的特征

In [21]:
# 加载库
import pandas as pd

# 创建数据帧
dataframe = pd.DataFrame()

# 创建数据
dataframe['prices'] = [1.1, 2.2, 3.3, 4.4, 5.5]

# 让值滞后一行
dataframe['previous_days_stock_price'] = dataframe['prices'].shift(1)

# 查看数据
dataframe
# 输出:
#    prices  previous_days_stock_price
# 0     1.1                        NaN
# 1     2.2                        1.1
# 2     3.3                        2.2
# 3     4.4                        3.3
# 4     5.5                        4.4

,prices,previous_days_stock_price
0,1.1,NaN
1,2.2,1.1
2,3.3,2.2
3,4.4,3.3
4,5.5,4.4


讨论：
滞后特征常用于时间序列分析
第一行会自动填充为NaN（缺失值）

# 7.8使用滚动时间窗口

问题描述：计算时间序列的滚动统计量

In [23]:
# 加载库
import pandas as pd

# 创建数据帧，设置索引
dataframe = pd.DataFrame()
dataframe['date'] = pd.date_range('1/1/2001', periods=5, freq='ME')
dataframe['stock_price'] = [1, 2, 3, 4, 5]
dataframe.set_index('date', inplace=True)

# 计算滚动平均值（窗口=3）
dataframe.rolling(window=3).mean()
# 输出:
#             stock_price
# date
# 2001-01-31         NaN
# 2001-02-28         NaN
# 2001-03-31         2.0
# 2001-04-30         3.0
# 2001-05-31         4.0

,stock_price
date,
2001-01-31,NaN
2001-02-28,NaN
2001-03-31,2.0
2001-04-30,3.0
2001-05-31,4.0


其他滚动统计量

In [28]:
# 滚动求和
dataframe.rolling(window=3).sum()


,stock_price
date,
2001-01-31,NaN
2001-02-28,NaN
2001-03-31,6.0
2001-04-30,9.0
2001-05-31,12.0


In [29]:

# 滚动最大值
dataframe.rolling(window=3).max()


,stock_price
date,
2001-01-31,NaN
2001-02-28,NaN
2001-03-31,3.0
2001-04-30,4.0
2001-05-31,5.0


In [30]:
# 滚动标准差
dataframe.rolling(window=3).std()


,stock_price
date,
2001-01-31,NaN
2001-02-28,NaN
2001-03-31,1.0
2001-04-30,1.0
2001-05-31,1.0


In [27]:
# 滚动相关系数
dataframe.rolling(window=3).corr()

,,stock_price
date,,
2001-01-31,stock_price,NaN
2001-02-28,stock_price,NaN
2001-03-31,stock_price,1.0
2001-04-30,stock_price,1.0
2001-05-31,stock_price,1.0


# 7.9 处理时间序列中的缺失值

问题描述：处理时间序列数据中的缺失值

In [32]:
# 加载库
import pandas as pd
import numpy as np

# 创建日期
dates = pd.date_range('1/1/2001', periods=5, freq='D')

# 创建带缺失数据的特征
dataframe = pd.DataFrame({'date': dates, 'sales': [1.0, 2.0, np.nan, np.nan, 5.0]})
dataframe.set_index('date', inplace=True)

# 线性插值
dataframe.interpolate()
# 输出:
#             sales
# date
# 2001-01-01    1.0
# 2001-01-02    2.0
# 2001-01-03    3.0
# 2001-01-04    4.0
# 2001-01-05    5.0


,sales
date,
2001-01-01,1.0
2001-01-02,2.0
2001-01-03,3.0
2001-01-04,4.0
2001-01-05,5.0


In [35]:
# 向前填充
dataframe.ffill()
# 输出: 用前一个值填充缺失值

,sales
date,
2001-01-01,1.0
2001-01-02,2.0
2001-01-03,2.0
2001-01-04,2.0
2001-01-05,5.0


In [36]:
# 向后填充
dataframe.bfill()
# 输出: 用后一个值填充缺失值

,sales
date,
2001-01-01,1.0
2001-01-02,2.0
2001-01-03,5.0
2001-01-04,5.0
2001-01-05,5.0


高级插值方法：

In [37]:
# 二次插值
dataframe.interpolate(method='quadratic')

# 指定插值限制（最多填充2个连续缺失）
dataframe.interpolate(limit=2)

,sales
date,
2001-01-01,1.0
2001-01-02,2.0
2001-01-03,3.0
2001-01-04,4.0
2001-01-05,5.0


# 第7章完整知识点总结
日期时间转换：
pd.to_datetime()：字符串转datetime，支持自定义格式
format参数：指定日期字符串格式（%Y-%m-%d %H:%M:%S）
errors='coerce'：错误值转换为NaT

时区处理：
dt.tz_localize()：设置时区
dt.tz_convert()：转换时区
pytz库：提供全球时区信息

日期筛选：
使用布尔条件：dataframe[(date > start) & (date <= end)]

特征提取：
dt.year/month/day/hour/minute：提取时间组件
dt.weekday_name/weekday：星期几名称/数字

时间差计算：
日期相减：df['end'] - df['start']
.dt.days：提取天数差


滞后特征：
.shift(n)：创建滞后n期的特征



滚动窗口：
.rolling(window=n)：创建滚动时间窗口
.mean()/sum()/max()/std()：计算滚动统计量


缺失值处理：
.interpolate()：线性插值
.ffill()：前向填充
.bfill()：后向填充
method='quadratic'：二次插值

最佳实践：
时间序列分析前确保日期列为datetime类型
设置合适的索引（DatetimeIndex）可提高处理效率
滚动窗口计算时注意前几项为NaN（窗口未满）
插值方法选择需根据数据特点决定（线性、二次、前向/后向）